<a href="https://colab.research.google.com/github/Hormymac/AI/blob/main/Tarea_PLN_EA_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Etiquetado Morfosintáctico (HMM)

Objetivos

Esta actividad consiste en implementar y aplicar un método basado en modelos ocultos de Markov (HMM) para realizar el etiquetado morfosintáctico de una oración.


In [6]:
# Se inicia con la importación de las librerías
# ------------------------------------------------------------------------------
# Cabe indicar que no está permitido usar librerías de NLP (como NLTK o Spacy o ninguna otra libreria) para la lógica del HMM.
# Se usará pandas y numpy solo para manejo de matrices y exportación a Excel como requerimiento para cumplir con la actividad.

import pandas as pd
import numpy as np
import os
import sys

In [7]:
# PASO No 1: CARGA Y PREPROCESAMIENTO DEL CORPUS (Corpus-tagged.txt)
# Construcción de las tablas de probabilidades para el etiquetador morfosintáctico
# En esta primera parte de la actividad se tiene que implementar en Python el etiquetador morfosintáctico basado en un HMM bigrama a partir de un corpus etiquetado.
# Para ello se debe usar el corpus Corpus-tagged, que se encuentra disponible como material adicional de la actividad que se encuntra en GitHub personal.
# ------------------------------------------------------------------------------

## DESCARGA DE DATOS
#url_corpus = "https://github.com/Hormymac/AI/blob/main/Corpus-tagged.txt"
url_corpus = "https://raw.githubusercontent.com/OscarJimenezFlores/PLN/refs/heads/main/Actividad_EPN/Corpus-tagged.txt"
archivo_corpus = "Corpus-tagged.txt"
os.system(f"wget -O {archivo_corpus} {url_corpus}")

# FUNCIÓN DE PARSEO
def parsear_corpus_a_dataframe(ruta_archivo):
    lista_datos = []
    prev_tag = "START" # Estado inicial ficticio para la primera frase

    try:
        with open(ruta_archivo, 'r', encoding='utf-8') as f:
            for linea in f:
                linea = linea.strip()

                # Si la línea está vacía, indica fin de frase en este formato de corpus habitual
                # Así que se resetea el tag previo a START
                if not linea:
                    prev_tag = "START"
                    continue

                if linea.startswith('<'): # Se salta metadatos xml
                    continue

                partes = linea.split()
                if len(partes) >= 3:
                    token = partes[0]
                    lema = partes[1]
                    tag = partes[2] # Usamos el TAG COMPLETO (EAGLES) según rúbrica
                    # Las etiquetas gramaticales (POS tags) utilizadas para anotar la
                    # información morfosintáctica del corpus son las definidas en FreeLing
                    # y se basan en EAGLES, una recomendación para la anotación de la mayoría
                    # de las lenguas europeas

                    lista_datos.append({
                        'Token': token,
                        'Token_Lower': token.lower(), # Se usa para buscar emisiones
                        'Tag': tag,
                        'Tag_Previo': prev_tag
                    })

                    # El tag actual se convierte en el previo para la siguiente iteración
                    prev_tag = tag

        return pd.DataFrame(lista_datos)

    except Exception as e:
        print(f"Error: {e}")
        return pd.DataFrame()

# Se ejecuta la carga
df_corpus = parsear_corpus_a_dataframe(archivo_corpus)

print(f"Corpus cargado con {len(df_corpus)} palabras.")
print("Vista previa con la columna 'Tag_Previo' calculada correctamente:")
display(df_corpus.head(10))



Corpus cargado con 492 palabras.
Vista previa con la columna 'Tag_Previo' calculada correctamente:


,Token,Token_Lower,Tag,Tag_Previo
0,Tristana,tristana,NP00000,START
1,es,es,VSIP3S0,NP00000
2,una,una,DI0FS0,VSIP3S0
3,película,película,NCFS000,DI0FS0
4,de,de,SPS00,NCFS000
5,el,el,DA0MS0,SPS00
6,director,director,NCMS000,DA0MS0
7,español,español,AQ0MS0,NCMS000
8,nacionalizado,nacionalizado,VMP00SM,AQ0MS0
9,mexicano,mexicano,AQ0MS0,VMP00SM


In [8]:
# CONTEO DE FRECUENCIAS:

# Se cuenta cuántas veces aparece cada Tag (Unigramas)
# Consecuentemente será el denominador en las probabilidades
conteo_tags = df_corpus['Tag'].value_counts().to_dict()
# Se añade el conteo de START para poder dividir en transiciones iniciales
conteo_tags['START'] = df_corpus[df_corpus['Tag_Previo']== 'START'].shape[0]

# Se cuenta las EMISIONES: Pares ( Tag -- Palabra )
# Se agrupa por Tag y token_lower para normalizar mayúsculas/minúsculas
df_emisiones_counts = df_corpus.groupby(['Tag','Token_Lower']).size().reset_index(name='Count')

# Se cuentan las transiciones: Pares (Tag_Previo -- Tag)
df_transiciones_counts = df_corpus.groupby(['Tag_Previo','Tag']).size().reset_index(name='Count')

# CÁLCULO DE PROBABILIDADES:
# Cálculo de las probabilidades de emisión del HMM a partir del corpus etiquetado. Construcción de la tabla de probabilidades de emisión.
prob_emision = {}
for index, row in df_emisiones_counts.iterrows():
    tag = row['Tag']
    word = row['Token_Lower']
    count =  row['Count']
    prob_emision[(tag, word)] = count / conteo_tags[tag]

# Diccionario para transiciones:
# Cálculo de las probabilidades de transición del HMM a partir del corpus etiquetado. Construcción de la tabla de probabilidades de transmisión
prob_transicion = {}
for index, row in df_transiciones_counts.iterrows():
    prev = row['Tag_Previo']
    curr = row['Tag']
    count =  row['Count']
    prob_transicion[(prev, curr)] = count / conteo_tags[tag]

print("Entrenamiento completado.")
print(f"Total pares de emisión: {len(prob_emision)}")
print(f"Total pares de transición: {len(prob_transicion)}")

# Se exporta a formato EXCEL
# Se prepara DataFrames limpios para poder exportar
# Con las probabilidades de emisión y las de transición, calculadas para todas las etiquetas y tokens (palabras) que aparecen en el corpus.

df_export_emision = pd.DataFrame([
    {'Tag': k[0], 'Palabra': k[1], 'Probabilidad':v}
    for k, v in prob_emision.items()
])

df_export_transicion = pd.DataFrame([
    {'Tag_Previo': k[0], 'Tag_Actual': k[1], 'Probabilidad':v}
    for k, v in prob_transicion.items()
])

with pd.ExcelWriter('1_Tablas_Probabilidades_1.xlsx') as writer:
    df_export_emision.to_excel(writer, sheet_name='Emision', index=False)
    df_export_transicion.to_excel(writer, sheet_name='Transicion', index=False)

print("Achivo '1_Tablas_Probabilidades_1.xlsx' fue generado. Descargar de la carpeta archivos")


Entrenamiento completado.
Total pares de emisión: 242
Total pares de transición: 288
Achivo '1_Tablas_Probabilidades_1.xlsx' fue generado. Descargar de la carpeta archivos


In [9]:
# Se verifica la carga total
print(f"Total de filas procesadas en memoria: {len(df_corpus)}")

# Se visualiza los diccionarios completos
print("\n" + "="*50)
print("TABLAS DE PROBABILIDADES DE TRANSICIÓN (Matriz A)")
print("="*50)

import pprint
pprint.pprint(prob_transicion)

print("\n" + "="*50)
print("TABLAS DE PROBABILIDADES DE EMISIÓN (Matriz B)")
print("="*50)
pprint.pprint(prob_emision)



Total de filas procesadas en memoria: 492

TABLAS DE PROBABILIDADES DE TRANSICIÓN (Matriz A)
{('AO0MS0', 'NCMS000'): 1.0,
 ('AQ0CS0', 'CC'): 2.0,
 ('AQ0CS0', 'CS'): 1.0,
 ('AQ0CS0', 'Fc'): 1.0,
 ('AQ0CS0', 'Fe'): 1.0,
 ('AQ0CS0', 'Fx'): 1.0,
 ('AQ0CS0', 'NCFS000'): 2.0,
 ('AQ0CS0', 'NCMN000'): 1.0,
 ('AQ0CS0', 'NCMS000'): 3.0,
 ('AQ0CS0', 'RG'): 1.0,
 ('AQ0CS0', 'SPS00'): 3.0,
 ('AQ0CS0', 'VMP00SM'): 1.0,
 ('AQ0FS0', 'CC'): 1.0,
 ('AQ0FS0', 'SPS00'): 1.0,
 ('AQ0MP0', 'CC'): 1.0,
 ('AQ0MS0', 'CS'): 1.0,
 ('AQ0MS0', 'Fc'): 1.0,
 ('AQ0MS0', 'NCMS000'): 1.0,
 ('AQ0MS0', 'NP00000'): 1.0,
 ('AQ0MS0', 'SPS00'): 2.0,
 ('AQ0MS0', 'VMP00SM'): 1.0,
 ('CC', 'AQ0CS0'): 1.0,
 ('CC', 'AQ0MS0'): 1.0,
 ('CC', 'CS'): 1.0,
 ('CC', 'NCFS000'): 1.0,
 ('CC', 'NCMS000'): 1.0,
 ('CC', 'NP00000'): 1.0,
 ('CC', 'P0000000'): 3.0,
 ('CC', 'PP3CNA00'): 1.0,
 ('CC', 'PP3CSD00'): 1.0,
 ('CC', 'RG'): 3.0,
 ('CC', 'SPS00'): 2.0,
 ('CC', 'VMIP3S0'): 3.0,
 ('CC', 'VMIS3S0'): 1.0,
 ('CC', 'VMP00SF'): 1.0,
 ('CC', 'VMP00S

In [10]:
# PASO NO 2. IMPLEMENTACIÓN DEL ALGORITMO DE VITERBI (ETIQUETADO)
# El algoriTmo sirve para decodificar la secuencia más probable
# Oración:  "El enfermo grave habla de trasplantes."

def algoritmo_viterbi(oracion_str, estados_posibles, p_emision, p_transicion):

    # 1. Preprocesamiento de la oración (tokenización simple)
    # Se separa el punto final si está pegado
    oracion_str = oracion_str.replace(".", " .")
    palabras = oracion_str.split()
    palabras_lower = [p.lower() for p in palabras]

    T = len(palabras)
    N = len(estados_posibles)

    # 2. Inicialización de estructuras
    # Viterbi Matrix: filas=Estados, col=Palabras. Se almacena la probabilidad máxima.
    viterbi_mat = np.zeros((N, T))

    # Backpointer Matrix: Se almacena el índice del estado previo que dio la probabilidad máxima.
    backpointer = np.zeros((N, T), dtype=int)

    # Mapeo de estados a índices para poder usar matrices numpy
    estado_a_idx = {estado: i for i, estado in enumerate(estados_posibles)}

    primera_palabra = palabras_lower[0]

    for s, estado in enumerate(estados_posibles):
        p_trans = p_transicion.get(('START', estado), 0)

        # Prob Emision: P(Palabra | Estado)
        p_emit = p_emision.get((estado, primera_palabra), 0)

        # Viterbi[s, 0]
        viterbi_mat[s, 0] = p_trans * p_emit
        backpointer[s, 0] = 0 # No hay anterior

    # PASO DE RECURSIÓN
    for t in range(1, T):
        palabra_actual = palabras_lower[t]

        for s, estado_actual in enumerate(estados_posibles):
            # Optimización: Si la palabra es imposible para este estado (emisión 0), se salta
            # Esto ahorra tiempo de cómputo
            p_emit = p_emision.get((estado_actual, palabra_actual), 0)
            if p_emit == 0:
                viterbi_mat[s, t] = 0
                continue

            # Se busca cual fue el mejor estado previo
            mejor_prob_trans = -1
            mejor_prev_idx = 0

            for s_prev, estado_prev in enumerate(estados_posibles):
                # Probabilidad acumulada anterior
                v_prev = viterbi_mat[s_prev, t-1]

                if v_prev > 0: # Solo se calcula si el camino anterior es viable
                    # Transición del previo al actual
                    trans = p_transicion.get((estado_prev, estado_actual), 0)

                    prob_camino = v_prev * trans

                    if prob_camino > mejor_prob_trans:
                        mejor_prob_trans = prob_camino
                        mejor_prev_idx = s_prev

            # Se asigna valores a las matrices
            if mejor_prob_trans > 0:
                viterbi_mat[s, t] = mejor_prob_trans * p_emit
                backpointer[s, t] = mejor_prev_idx
            else:
                viterbi_mat[s, t] = 0

    # --- PASO DE TERMINACIÓN Y BACKTRACKING ---

    # 1. Encontrar la mejor probabilidad en la última columna
    mejor_prob_final = np.max(viterbi_mat[:, T-1])
    mejor_last_idx = np.argmax(viterbi_mat[:, T-1])

    # 2. Reconstruir el camino hacia atrás
    mejor_camino_indices = [mejor_last_idx]

    for t in range(T-1, 0, -1):
        idx_actual = mejor_camino_indices[-1]
        idx_previo = backpointer[idx_actual, t]
        mejor_camino_indices.append(idx_previo)

    # Se invierte porque se fue hacia atrás
    mejor_camino_indices.reverse()

    # Se mapea índices a etiquetas de texto
    mejor_secuencia_tags = [estados_posibles[i] for i in mejor_camino_indices]

    # SE empaqueta resultados
    resultados = []
    for word, tag in zip(palabras, mejor_secuencia_tags):
        resultados.append((word, tag))

    return viterbi_mat, resultados, palabras

# EJECUCIÓN

# Se obtiene la lista única de estados (tags) del corpus
lista_estados = df_corpus['Tag'].unique().tolist()
oracion_test = "El enfermo grave habla de trasplantes."

matriz_v, etiquetas_finales, tokens = algoritmo_viterbi(
    oracion_test,
    lista_estados,
    prob_emision,
    prob_transicion
)

#  RESULTADOS EN PANTALLA
print("\n--- RESULTADO DEL ETIQUETADO ---")
for palabra, tag in etiquetas_finales:
    print(f"{palabra:15} -> {tag}")

# EXPORTACIÓN MATRIZ VITERBI A EXCEL
# Con la matriz de probabilidades de la ruta Viterbi para el etiquetado morfosintáctico de la oración «Habla con el enfermo grave de trasplantes.».
# Se convierte la matriz numpy a DataFrame para guardar
# Como hay muchos estados, se filtra solo los que tienen alguna probabilidad > 0 para que el Excel sea legible
df_viterbi = pd.DataFrame(matriz_v, index=lista_estados, columns=tokens)
df_viterbi_limpia = df_viterbi.loc[(df_viterbi!=0).any(axis=1)] # Solo filas con valores

with pd.ExcelWriter('2_Matriz_Viterbi_2.xlsx') as writer:
    df_viterbi_limpia.to_excel(writer, sheet_name='Viterbi')

print("\nArchivo '2_Matriz_Viterbi_2.xlsx' generado.")








--- RESULTADO DEL ETIQUETADO ---
El              -> NP00000
enfermo         -> NP00000
grave           -> NP00000
habla           -> NP00000
de              -> NP00000
trasplantes     -> NP00000
.               -> NP00000

Archivo '2_Matriz_Viterbi_2.xlsx' generado.
